In [1]:
%%capture
!pip install openai==0.28.1

In [3]:
import re,ast, os
import openai
import numpy as np
import time
!cp -r '/content/drive/MyDrive/OntologyDevelopment' '/content'
from OntologyDevelopment.CQbyCQ import cqbycq_prompt
from OntologyDevelopment.stories import Datasets
from OntologyDevelopment.Zeroshot import zero_shot
from OntologyDevelopment.waterfall import DecomposedPromtingWaterfall

def chat_with_gpt(prompt_text,temp = 0,penalty = 0):
    # print(prompt_text)
    # print('\n\n\n')
    # input()
    # return open('input.txt').read()
    openai.api_type = ""
    openai.api_base = ""
    openai.api_version = ""
    openai.api_key = ""
    response = openai.ChatCompletion.create(
      engine=  "",
      messages = [{"role":"system","content":prompt_text}],
      temperature=temp,
      # max_tokens=20000, #*********
      top_p=0,
      frequency_penalty=penalty,
      presence_penalty=penalty,
      stop=None)
    return response['choices'][0]['message']['content']

def write_to_log(message, logfile='logfile.log', separator='**********************************\n\n*************************\n'):
    with open(logfile, 'a') as f:
        f.write(message + '\n')
        f.write(separator)
def delete_logfile():
  try:
    file_path = "logfile.log"
    if os.path.exists(file_path):
        os.remove(file_path)
  except:
    pass
def find_last_step(text):
    steps = re.findall(r'Step (\d+):', text)
    return int(steps[-1]) if steps else 10

# cq by cq

In [ ]:
def cqbycq(story,cqs,sleep = 3):
  rdf = ''
  for i in range(len(cqs)):
    print('doing cq #',i+1)
    rdf = chat_with_gpt(cqbycq_prompt.format(story = story,CQ = cqs[i],rdf = rdf))
    write_to_log(rdf)
    if i < len(cqs) - 1:
      time.sleep(sleep)
  write_to_log('final output of cqbycq')
  write_to_log(rdf)
  return rdf

In [ ]:
stories = Datasets()

write_to_log('toy story 1')
S1,CQs1 = stories.toystory1, stories.toystory1_cqs
cqbycq(S1,CQs1,sleep = 0)

write_to_log('toy story 2')
S1,CQs1 = stories.toystory2, stories.toystory2_cqs
cqbycq(S1,CQs1,sleep = 0)


write_to_log('toy story 3')
S1,CQs1 = stories.toystory3, stories.toystory3_cqs
cqbycq(S1,CQs1,sleep = 10)

# Decomposed Prompting:
## ZeroShot

In [ ]:
stories = Datasets()

write_to_log('toy story 1')
S1,CQs1 = stories.toystory1, stories.toystory1_cqs
res = chat_with_gpt(zero_shot.format(story = S1+'competency questions are:\n'+'\n'.join(CQs1)))
write_to_log(res)


write_to_log('toy story 2')
S1,CQs1 = stories.toystory2, stories.toystory2_cqs
res = chat_with_gpt(zero_shot.format(story = S1+'competency questions are:\n'+'\n'.join(CQs1)))
write_to_log(res)


write_to_log('toy story 3')
S1,CQs1 = stories.toystory3, stories.toystory3_cqs
res = chat_with_gpt(zero_shot.format(story = S1+'competency questions are:\n'+'\n'.join(CQs1)))
write_to_log(res)


## waterfall

In [ ]:
stories = Datasets()

write_to_log('toy story 1')
S1,CQs1 = stories.toystory3, stories.toystory3_cqs

waterfall = DecomposedPromtingWaterfall()
answer = chat_with_gpt(waterfall.prompt(1,story=S1,cqs=CQs1))
write_to_log(answer);time.sleep(5)
rdf    = chat_with_gpt(waterfall.prompt(2,story=S1,cqs=CQs1,previous_answer=answer))
write_to_log(rdf);time.sleep(5)
rdf    = chat_with_gpt(waterfall.prompt(3,rdf=rdf,story=S1,cqs=CQs1,previous_answer=answer))
write_to_log(rdf);time.sleep(5)
rdf    = rdf + chat_with_gpt(waterfall.prompt(4,rdf=rdf,story=S1,cqs=CQs1,previous_answer=answer))
write_to_log(rdf);time.sleep(5)
rdf    = rdf + chat_with_gpt(waterfall.prompt(5,rdf=rdf,story=S1,cqs=CQs1,previous_answer=answer))
write_to_log(rdf)


# Chain of Thoughts


In [ ]:
stories = Datasets()
S1,CQs1 = stories.toystory1, stories.toystory1_cqs
S2,CQs2 = stories.toystory2, stories.toystory2_cqs
S3,CQs3 = stories.toystory3, stories.toystory3_cqs

prompt_text = '''
Your task is to contribute in creation of a well-structured ontology informations that appeared in the given story, requirements, and restrictions (if there are any). You should only provide steps to solve this task not actually solving it. Imagin yourself as a prompter of chatGPT and we want to use chain of thoughts. So in this step only create a plan or steps to solve the task. the way you approach this is that you create only a code block in the output that contains a list of strings. Each string in the list is a plan or thought. the first string is the first thought and so on. do not output any other comment or explanation. here are some definisions of ontology components to help you understand the problem in a better way:
Classes are the keywords/classes that are going to be node types in the knowledge graph ontology. try to extract all classes, in addition, classes are also can be defined for reification. We use Turtle Syntax for representation. Hierarchies are rdfs:subClassOf in the turtle syntax. They can be used to classify similar classes in one superclass. To do this you can find similar nodes and create/use a class as their parent class, for example, adding the node "Cl_employee" is a good middleware and superclass for "Cl_Professors" and "Cl_Administrator" if the story is about modeling ontology of a university. Mostly the lengthier the hierarchy the better. One way can be categorizing classes into several classes and creating superclasses for them. Important: Class names have Cl_ as the prefix for example Cl_Professors. Also keep in mind you can add Equivalent To, General class axioms, Disjoint with, and Disjoint Union of, for each class.
In your ontology modeling, for each competency question, when faced with complex scenarios that involve more than two entities or a combination of entities and datatypes, apply reification. Specifically, create a pivot class to act as an intermediary for these entities, ensuring the nuanced relationships are accurately captured. For instance, when representing "a user accessed a resource at a given time", establish a pivot class like Cl_UserResourceUsage, linked from the user, resource, and the specific time of access to Cl_UserResourceInteraction, rather than directly connecting the user to both the resource and time.
Then you need to create properties (owl:Property). In this step, you use classes from the previous stage and create object and data properties to connect them and establish the ontology. Always output a turtle syntax, if you need more classes to model a competency question between more than 2 concepts, feel free to add more pivot (reification) classes here. try to find as much relation as possible by reading competency questions, restrictions, and stories. At this stage, you can create both data and object properties. Data properties are between classes or hierarchy classes and data types such as xsd:string, xsd:integer, xsd:decimal, xsd:dateTime, xsd:date, xsd:time, xsd:boolean, xsd:byte, xsd:double, xsd:float and etc. For example, in the university domain, we have: employee_id a owl:Property ; rdfs:domain :cl_teacher ; rdfs:range xsd:integer. Object properties are between classes. try to find as much relation as possible by reading competency questions and the story. Feel free to use rdfs:subPropertyOf for creating hierarchies for relations. For modeling properties (object or data properties) if it is necessary, use these relations characteristics: Functional, Inverse functional, Transitive, Symmetric, Asymmetric, Reflexive, and Irreflexive. Also, you are flexible in domain and range so you can use Cl_class1 or Cl_class2 in domain and range or disjoint with, the inverse of between relations.
Your output at this stage is a plan. like step 1, step 2, ... step n (max 10 steps). dont create the ontology at this stage. after this stage i will give you 'next' in the chat and you create a code plus a small plan for the next step. when the last step executed, the final output be the final owl ontology that models the story with its restirictions and be able to answer the competency questions. Plan steps must be in a python list of string like: ['step1','step2',...]
these are the prifixes:
@prefix : <http://www.example.org/test#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
Important: your output should be only owl turtle. Don't write any explanation before the code block or after. Also the code must have 0 comments.
end of footer
here is one story to know how they look like:
{story}
'''

In [ ]:
delete_logfile()


def CoT(prompt_text,S,CQs,short_it=False):
  prompt_text= prompt_text.format(story=S+'\n'.join(CQs))
  write_to_log('chain of thoughts')
  plan = chat_with_gpt(prompt_text)
  write_to_log(str(plan))
  History = prompt_text + '\n GPT:' + str(plan) + 'Prompt:\n'
  plan_len = find_last_step(plan)
  for i in range(plan_len):
    print('step in process: ',i+1)
    next_promt = f'now perform only step {i+1} of this plan by providing its code (turtule syntax)'
    History = History + next_promt
    time.sleep(5)
    #### short it for gpt3? ###
      #if the context is long for the model, we short it here, to be implemented ...
    #######################
    respond = chat_with_gpt(History)
    write_to_log(respond)
    History = History + '\n GPT:'+ respond + 'Prompt:\n'
  respond = chat_with_gpt(History+'\n Now give the final output of this plan:')
  write_to_log(str(respond))
  return respond

# rdf1 = CoT(prompt_text,S1,CQs1)
# rdf2 = CoT(prompt_text,S2,CQs2)
# rdf3 = CoT(prompt_text,S3,CQs3)


## Main Experiment

In [ ]:
stories = Datasets()
S,CQs = stories.story_theatre,stories.story_theatre_cqs
delete_logfile()

write_to_log('hospital_story')


rdf1 = CoT(prompt_text,S,CQs)
rdf2 = CoT(prompt_text,S,CQs)
rdf3 = CoT(prompt_text,S,CQs)


step in process:  1
step in process:  2
step in process:  3
step in process:  4
step in process:  5
step in process:  6
step in process:  7
step in process:  8
step in process:  9
step in process:  10
step in process:  1
step in process:  2
step in process:  3
step in process:  4
step in process:  5
step in process:  6
step in process:  7
step in process:  8
step in process:  9
step in process:  10
step in process:  1
step in process:  2
step in process:  3
step in process:  4
step in process:  5
step in process:  6
step in process:  7
step in process:  8
step in process:  9
step in process:  10


In [ ]:
rdf1

'```turtle\n@prefix : <http://www.example.org/test#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n\n:Cl_Theatre a rdfs:Class .\n:Cl_Festival a rdfs:Class .\n:Cl_Play a rdfs:Class .\n:Cl_Actor a rdfs:Class .\n:Cl_Role a rdfs:Class .\n:Cl_Production a rdfs:Class ;\n    rdfs:subClassOf [\n        a owl:Restriction ;\n        owl:onProperty :hasPremier ;\n        owl:cardinality "1"^^xsd:nonNegativeInteger\n    ] .\n:Cl_City a rdfs:Class .\n:Cl_Country a rdfs:Class .\n:Cl_Author a rdfs:Class .\n:Cl_Newspaper a rdfs:Class .\n:Cl_Review a rdfs:Class .\n\n:Cl_TheatreInstitute a rdfs:Class ;\n    rdfs:subClassOf :Cl_Theatre .\n:Cl_GrandTheatre a rdfs:Class ;\n    rdfs:subClassOf :Cl_Theatre .\n:Cl_ItalianActor a rdfs:Class ;\n    rdfs:subClassOf :Cl_Actor .\n:Cl_GuestActor a rdfs:Class ;\n    rdfs:subClassOf :Cl_Actor 

In [ ]:
rdf2

'```turtle\n@prefix : <http://www.example.org/test#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n\n:Cl_Theatre a rdfs:Class .\n:Cl_Festival a rdfs:Class .\n:Cl_Play a rdfs:Class .\n:Cl_Actor a rdfs:Class .\n:Cl_Role a rdfs:Class .\n:Cl_Production a rdfs:Class ;\n    rdfs:subClassOf [\n        a owl:Restriction ;\n        owl:onProperty :hasPremier ;\n        owl:cardinality "1"^^xsd:nonNegativeInteger\n    ] .\n:Cl_Review a rdfs:Class .\n:Cl_City a rdfs:Class .\n:Cl_Country a rdfs:Class .\n:Cl_GrandTheatre a rdfs:Class ;\n    rdfs:subClassOf :Cl_Theatre .\n:Cl_PisaTheatre a rdfs:Class ;\n    rdfs:subClassOf :Cl_Theatre .\n:Cl_VeniceTheatre a rdfs:Class ;\n    rdfs:subClassOf :Cl_Theatre .\n:Cl_ActorRole a rdfs:Class .\n\n:hasLocation a rdf:Property ;\n    rdfs:domain :Cl_Theatre ;\n    rdfs:range :Cl_City .\n:

In [ ]:
rdf3

'```turtle\n@prefix : <http://www.example.org/test#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n\n:Cl_Theatre a rdfs:Class .\n:Cl_Festival a rdfs:Class .\n:Cl_Play a rdfs:Class .\n:Cl_Actor a rdfs:Class .\n:Cl_Role a rdfs:Class .\n:Cl_Production a rdfs:Class .\n:Cl_Review a rdfs:Class .\n:Cl_City a rdfs:Class .\n:Cl_Country a rdfs:Class .\n\n:Cl_Theatre rdfs:subClassOf :Cl_City .\n:Cl_Festival rdfs:subClassOf :Cl_City .\n:Cl_Play rdfs:subClassOf :Cl_Production .\n:Cl_Actor rdfs:subClassOf :Cl_Role .\n\n:hasLocation a rdf:Property ;\n    rdfs:domain :Cl_Theatre ;\n    rdfs:range xsd:string .\n:hasPlay a rdf:Property ;\n    rdfs:domain :Cl_Production ;\n    rdfs:range :Cl_Play .\n:hasActor a rdf:Property ;\n    rdfs:domain :Cl_Production ;\n    rdfs:range :Cl_Actor .\n:hasRole a rdf:Property ;\n    rdfs:domain 

# Tree of Thoughts

In [ ]:
stories = Datasets()
S1,CQs1 = stories.toystory1, stories.toystory1_cqs
S2,CQs2 = stories.toystory2, stories.toystory2_cqs
S3,CQs3 = stories.toystory3, stories.toystory3_cqs
S,CQs = stories.story_theatre,stories.story_theatre_cqs

Story = S3+'\n'.join(CQs3)

treeOfT='''Your task is to contribute in creation of a well-structured ontology informations that appeared in the given story, requirements, and restrictions (if there are any). You should only provide few diffierent plans that each contains steps to solve this task not actually solving it. Imagin yourself as a prompter of chatGPT and we want to use tree of thoughts. So in this step only create few plans with steps to solve the task. the way you approach this is that you create only a code block in the output that contains a list of  list of strings. Each list is a different plan to solve the problem from step 1 to end, and each string in every list is a thought. the first string is the first thought/step and so on. do not output any other comment or explanation. here are some definisions of ontology components to help you understand the problem in a better way:
CClasses are the keywords/classes that are going to be node types in the knowledge graph ontology. try to extract all classes, in addition, classes are also can be defined for reification. We use Turtle Syntax for representation. Hierarchies are rdfs:subClassOf in the turtle syntax. They can be used to classify similar classes in one superclass. To do this you can find similar nodes and create/use a class as their parent class, for example, adding the node "Cl_employee" is a good middleware and superclass for "Cl_Professors" and "Cl_Administrator" if the story is about modeling ontology of a university. Mostly the lengthier the hierarchy the better. One way can be categorizing classes into several classes and creating superclasses for them. Important: Class names have Cl_ as the prefix for example Cl_Professors. Also keep in mind you can add Equivalent To, General class axioms, Disjoint with, and Disjoint Union of, for each class.
In your ontology modeling, for each competency question, when faced with complex scenarios that involve more than two entities or a combination of entities and datatypes, apply reification. Specifically, create a pivot class to act as an intermediary for these entities, ensuring the nuanced relationships are accurately captured. For instance, when representing "a user accessed a resource at a given time", establish a pivot class like Cl_UserResourceUsage, linked from the user, resource, and the specific time of access to Cl_UserResourceInteraction, rather than directly connecting the user to both the resource and time.
Then you need to create properties (owl:Property). In this step, you use classes from the previous stage and create object and data properties to connect them and establish the ontology. Always output a turtle syntax, if you need more classes to model a competency question between more than 2 concepts, feel free to add more pivot (reification) classes here. try to find as much relation as possible by reading competency questions, restrictions, and stories. At this stage, you can create both data and object properties. Data properties are between classes or hierarchy classes and data types such as xsd:string, xsd:integer, xsd:decimal, xsd:dateTime, xsd:date, xsd:time, xsd:boolean, xsd:byte, xsd:double, xsd:float and etc. For example, in the university domain, we have: employee_id a owl:Property ; rdfs:domain :cl_teacher ; rdfs:range xsd:integer. Object properties are between classes. try to find as much relation as possible by reading competency questions and the story. Feel free to use rdfs:subPropertyOf for creating hierarchies for relations. For modeling properties (object or data properties) if it is necessary, use these relations characteristics: Functional, Inverse functional, Transitive, Symmetric, Asymmetric, Reflexive, and Irreflexive. Also, you are flexible in domain and range so you can use Cl_class1 or Cl_class2 in domain and range or disjoint with, the inverse of between relations.

Your output at this stage is a list of three different plans (make sure they solve the problem with different prespectives). Each plan contain a list like step 1, step 2, ... step n. dont create the ontology at this stage. after this stage i will give you 'plan x, next' in the chat and you perform the next step from plan x only (dont pay attention to the other plans). create a code plus a small plan for the next step. when the last step executed, the final output be the final owl ontology that models the story with its restirictions and be able to answer the competency questions. We will later compare the results of the owl files with 'merge' commands at the end (at this stage you need to see them and create a one owl file based on them, it would be like you see the answers of few different ontologist and based on your opinion pick or craete one from them), so dont compare untill the command.
these are the prifixes:
@prefix : <http://www.example.org/test#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
Important: your output should be only owl turtle. Don't write any explanation before the code block or after. Also the code must have 0 comments.
end of footer
Common mistake:
1- plans are identical, try to think differently when generating plans. For producing different plans, you can take three of these approaches and create plans based on them:Bottom-Up Plan, Top-Down Plan,,Agile Plan, Waterfall Plan, Incremental Plan, Spiral Plan, Prototype Plan, Evolutionary Plan, RAD (Rapid Application Development) Plan, Scrum Plan
2- important: when i ask to execute plan x, step y, your output at executing steps must be only codes not explanations.
3- producing output not based on the story and its competency questions
here is one story to know what they look like (The final ontology is about this story and its competency questions):
{story}
End of story

'''

In [ ]:
delete_logfile()
plans = chat_with_gpt(treeOfT.format(story=Story),temp = 0.5,penalty=0.5)
# plans = chat_with_gpt(treeOfT.format(story=S+'\n'.join(CQs)))
print(plans)

# plans

[
  [
    "Plan 1:",
    "Step 1: Identify the main classes in the story",
    "Step 2: Create a hierarchy of classes based on their relationships",
    "Step 3: Define object properties to connect the classes",
    "Step 4: Define data properties for additional information",
    "Step 5: Add restrictions and characteristics to the properties"
  ],
  [
    "Plan 2:",
    "Step 1: Identify the main classes in the story",
    "Step 2: Create a hierarchy of classes based on their relationships",
    "Step 3: Define object properties to connect the classes",
    "Step 4: Define data properties for additional information",
    "Step 5: Add restrictions and characteristics to the properties"
    
  
],
[
  	"Plan3:",
  	"step1 : identify all possible classess from given story.",
  	"step2 : create hierarchies between similar classess.",
  	"step3 : define object property between related classess.",
  	"step4 : define data property for additional information.",
  	"step5 : add restrictions an

In [ ]:
results = []
delete_logfile()
count_error = 0
len_plan = [5,5,5] 1/0
for i in [0,1,2]:
  print('plan in process: ',i+1)
  plan = plans[i]
  History = treeOfT.format(story=Story) + '\nNow here are the plans you created, in the following, we ask you which step to perform' +\
   'GPT output (plans and steps):' +  '\nplan 1: ' + '\n'.join(plans[0]) + '\nplan 2: ' + '\n'.join(plans[1]) + '\nplan 3: ' + '\n'.join(plans[2])
  for j in range(len_plan[i]):
    try: #if any error happend



      print(f'plan in process: {i+1} step #{j+1}')
      time.sleep(5)
      History += f'\nUser command: Now perform/execute plan {i+1}, step {j+1} by providing the code (turtle syntax) of this step (you must output a code only)'
      output = chat_with_gpt(History)
      # print(output)
      History = History + '\nGPT:'+output+'\n'
      write_to_log(output)



    except:
      print('some error, we try again')
      count_error += 1
      if count_error > 2:
        1/0
      j = j - 1
      continue

  output = chat_with_gpt(History+f'\nUser command: Now what is the final code output of plan {i+1}  (use turtle syntax and combine all steps into one output), dont miss anything')
  results.append(output)
  write_to_log(History)
  write_to_log(output)


plan in process:  1
plan in process: 1 step #1
plan in process: 1 step #2
plan in process: 1 step #3
plan in process: 1 step #4
plan in process: 1 step #5
plan in process:  2
plan in process: 2 step #1
plan in process: 2 step #2
plan in process: 2 step #3
plan in process: 2 step #4
plan in process: 2 step #5
plan in process:  3
plan in process: 3 step #1
plan in process: 3 step #2
plan in process: 3 step #3
plan in process: 3 step #4
plan in process: 3 step #5


In [ ]:
print('*******************\noutput of plan 1:')
print(results[0])
print('*******************\noutput of plan 2:')
print(results[1])
print('*******************\noutput of plan 3:')
print(results[2])

*******************
output of plan 1:
@prefix : <http://www.example.org/test#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# Step 1: Define classes
:Cl_TheatreFestival a rdfs:Class .
:Cl_Play a rdfs:Class .
:Cl_Location a rdfs:Class .
:Cl_Performance a rdfs:Class .

# Step 2: Define properties
:hasPlay a owl:ObjectProperty ;
    rdfs:domain :Cl_TheatreFestival ;
    rdfs:range :Cl_Play .

:hasLocation a owl:ObjectProperty ;
    rdfs:domain :Cl_TheatreFestival ;
    rdfs:range :Cl_Location .

:hasPerformance a owl:ObjectProperty ;
    rdfs:domain :Cl_TheatreFestival ;
    rdfs:range :Cl_Performance .

# Step 3: Define data properties
:duration a owl:DatatypeProperty ;
    rdfs:domain :Cl_TheatreFestival ;
    rdfs:range xsd:integer .

# Step 4: Define hierarchy
:Cl_LargeFestival rdfs:subClassOf :Cl_TheatreFestival .

# Step 5: Define restrictions
:Cl_LargeFestiv

In [ ]:
results_string = '\noutput of plan 1:\n' + results[0] + '\noutput of plan 2:\n' + results[1] + '\noutput of plan 3:\n' + results[2]
choose = treeOfT.format(story=Story) + '\nNow here are the plans you created, in the following, we show output of each plan you created in a list of output' + str(plans) +\
  'now you need to pick the one that suits the story and its requierment. just write why and then tell which one is the best.\n output of plans:\GPT: ' + results_string
chat_with_gpt(choose)


'I would choose plan 3 as the best plan because it includes all the necessary steps to model the ontology based on the story and its requirements. It identifies the main classes, creates hierarchies, defines object and data properties, adds restrictions and characteristics, and even includes competency questions and their answers. It covers all the essential aspects of ontology modeling for this particular story.'

## Mian experiment


In [ ]:
stories = Datasets()
S,CQs = stories.story_theatre,stories.story_theatre_cqs
delete_logfile()
Story = S+ '\n'.join(CQs)
write_to_log('hospital_story')
plans = chat_with_gpt(treeOfT.format(story=Story),temp = 0.5,penalty=0.5)
print(plans)

["Plan 1: Bottom-Up Plan",
 "Step 1: Identify the main classes from the story (e.g., Theatre, Festival, Play, Production, Actor)",
 "Step 2: Identify subclasses for each main class based on the story details (e.g., Theatre can have subclasses like GrandTheatre and TheatreInstitute)",
 "Step 3: Define properties for each class and subclass (e.g., Festival has properties like name, location, date)",
 "Step 4: Establish relationships between classes using object properties (e.g., Festival is related to Play through a 'features' property)",
 "Step 5: Create data properties for specific attributes of classes (e.g., Play has a 'writtenDuring' data property with range xsd:date)",
 "Step 6: Apply restrictions based on the story requirements (e.g., A production has exactly one premier)"],

["Plan 2: Top-Down Plan",
 "Step 1: Define a superclass for all entities in the story (e.g., TheatreEntity)",
 "Step 2: Create subclasses for this superclass based on the different types of entities mentioned

In [ ]:
results = []
delete_logfile()
count_error = 0
len_plan = [6,6,4] 1/0 #manually put size of each plan
for i in [0,1,2]:
  print('plan in process: ',i+1)
  plan = plans[i]
  History = treeOfT.format(story=Story) + '\nNow here are the plans you created, in the following, we ask you which step to perform' +\
   'GPT output (plans and steps):' +  '\nplan 1: ' + '\n'.join(plans[0]) + '\nplan 2: ' + '\n'.join(plans[1]) + '\nplan 3: ' + '\n'.join(plans[2])
  for j in range(len_plan[i]):
    try: #if any error happend
      print(f'plan in process: {i+1} step #{j+1}')
      time.sleep(5)
      History += f'\nUser command: Now perform/execute plan {i+1}, step {j+1} by providing the code (turtle syntax) of this step (you must output a code only)'
      output = chat_with_gpt(History)
      # print(output)
      History = History + '\nGPT:'+output+'\n'
      write_to_log(output)
    except:
      print('some error, we try again')
      count_error += 1
      if count_error > 2:
        1/0
      j = j - 1
      continue
  output = chat_with_gpt(History+f'\nUser command: Now what is the final code output of plan {i+1}  (use turtle syntax and combine all steps into one output), dont miss anything')
  results.append(output)
  write_to_log(History)
  write_to_log(output)


plan in process:  1
plan in process: 1 step #1
plan in process: 1 step #2
plan in process: 1 step #3
plan in process: 1 step #4
plan in process: 1 step #5
plan in process: 1 step #6
plan in process:  2
plan in process: 2 step #1
plan in process: 2 step #2
plan in process: 2 step #3
plan in process: 2 step #4
plan in process: 2 step #5
plan in process: 2 step #6
plan in process:  3
plan in process: 3 step #1
plan in process: 3 step #2
plan in process: 3 step #3
plan in process: 3 step #4


In [ ]:

results[0]+'\n**********************\n'+results[1]+'\n**********************\n'+results[2]

'@prefix : <http://www.example.org/test#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n:Cl_Theatre a rdfs:Class .\n:Cl_Festival a rdfs:Class .\n:Cl_Play a rdfs:Class .\n:Cl_Production a rdfs:Class .\n:Cl_Actor a rdfs:Class .\n:Cl_Role a rdfs:Class .\n:Cl_City a rdfs:Class .\n:Cl_Country a rdfs:Class .\n:Cl_Review a rdfs:Class .\n:Cl_Newspaper a rdfs:Class .\n\n:Cl_Theatre rdfs:subClassOf :Cl_City .\n:Cl_Festival rdfs:subClassOf :Cl_City .\n:Cl_Production rdfs:subClassOf :Cl_Theatre .\n:Cl_Actor rdfs:subClassOf :Cl_Production .\n:Cl_Role rdfs:subClassOf :Cl_Actor .\n:Cl_City rdfs:subClassOf :Cl_Country .\n:Cl_Review rdfs:subClassOf :Cl_Newspaper .\n\n:hasTheatre a rdf:Property ; rdfs:domain :Cl_City ; rdfs:range :Cl_Theatre .\n:hasFestival a rdf:Property ; rdfs:domain :Cl_City ; rdfs:range :Cl_Festival .\n:hasProduction a rdf:Property ; rdfs:domain :Cl_Theat

In [ ]:
results_string = '\noutput of plan 1:\n' + results[0] + '\noutput of plan 2:\n' + results[1] + '\noutput of plan 3:\n' + results[2]
choose = treeOfT.format(story=Story) + '\nNow here are the plans you created, in the following, we show output of each plan you created in a list of output' + str(plans) +\
  'now you need to pick the one that suits the story and its requierment. just write why and then tell which one is the best.\n output of plans:\GPT: ' + results_string
chat_with_gpt(choose)


"The best plan for this story and its requirements is Plan 2: Top-Down Plan. This plan starts by defining a superclass for all entities in the story, which is a good approach for this story as it involves a variety of entities such as Theatre, Festival, Play, Production, Actor, etc. The plan then proceeds to create subclasses for this superclass based on the different types of entities mentioned in the story. This approach allows for a clear and organized structure of the ontology. The plan also includes defining object and data properties to establish relationships and capture specific attributes of these classes, which is crucial for answering the competency questions. Finally, the plan applies restrictions as per the requirements mentioned in the story, ensuring that the ontology accurately represents the story's constraints."

# Graph of thoughts

In [ ]:
stories = Datasets()
S1,CQs1 = stories.toystory1, stories.toystory1_cqs
S2,CQs2 = stories.toystory2, stories.toystory2_cqs
S3,CQs3 = stories.toystory3, stories.toystory3_cqs
S,CQs = stories.story_theatre,stories.story_theatre_cqs
Story = S3+'\n'.join(CQs3)

graph_ofT = '''Your task is to contribute in creation of a well-structured ontology informations that appeared in the given story, requirements, and restrictions (if there are any). You should only provide few diffierent plans that each contains steps to solve this task not actually solving it. Imagin yourself as a prompter of chatGPT and we want to use tree of thoughts. So in this step only create few plans with steps to solve the task. the way you approach this is that you create only a code block in the output that contains a list of  list of strings. Each list is a different plan to solve the problem from step 1 to end, and each string in every list is a thought. the first string is the first thought/step and so on. do not output any other comment or explanation. here are some definisions of ontology components to help you understand the problem in a better way:
CClasses are the keywords/classes that are going to be node types in the knowledge graph ontology. try to extract all classes, in addition, classes are also can be defined for reification. We use Turtle Syntax for representation. Hierarchies are rdfs:subClassOf in the turtle syntax. They can be used to classify similar classes in one superclass. To do this you can find similar nodes and create/use a class as their parent class, for example, adding the node "Cl_employee" is a good middleware and superclass for "Cl_Professors" and "Cl_Administrator" if the story is about modeling ontology of a university. Mostly the lengthier the hierarchy the better. One way can be categorizing classes into several classes and creating superclasses for them. Important: Class names have Cl_ as the prefix for example Cl_Professors. Also keep in mind you can add Equivalent To, General class axioms, Disjoint with, and Disjoint Union of, for each class.
In your ontology modeling, for each competency question, when faced with complex scenarios that involve more than two entities or a combination of entities and datatypes, apply reification. Specifically, create a pivot class to act as an intermediary for these entities, ensuring the nuanced relationships are accurately captured. For instance, when representing "a user accessed a resource at a given time", establish a pivot class like Cl_UserResourceUsage, linked from the user, resource, and the specific time of access to Cl_UserResourceInteraction, rather than directly connecting the user to both the resource and time.
Then you need to create properties (owl:Property). In this step, you use classes from the previous stage and create object and data properties to connect them and establish the ontology. Always output a turtle syntax, if you need more classes to model a competency question between more than 2 concepts, feel free to add more pivot (reification) classes here. try to find as much relation as possible by reading competency questions, restrictions, and stories. At this stage, you can create both data and object properties. Data properties are between classes or hierarchy classes and data types such as xsd:string, xsd:integer, xsd:decimal, xsd:dateTime, xsd:date, xsd:time, xsd:boolean, xsd:byte, xsd:double, xsd:float and etc. For example, in the university domain, we have: employee_id a owl:Property ; rdfs:domain :cl_teacher ; rdfs:range xsd:integer. Object properties are between classes. try to find as much relation as possible by reading competency questions and the story. Feel free to use rdfs:subPropertyOf for creating hierarchies for relations. For modeling properties (object or data properties) if it is necessary, use these relations characteristics: Functional, Inverse functional, Transitive, Symmetric, Asymmetric, Reflexive, and Irreflexive. Also, you are flexible in domain and range so you can use Cl_class1 or Cl_class2 in domain and range or disjoint with, the inverse of between relations.

Your output at this stage is a list of three different plans (a 2D list) (make sure they solve the problem with different prespectives). Each plan contain a list like [step 1:, step 2:, ... step n:]. dont create the ontology at this stage. after this stage i will give you 'plan x, next' in the chat and you perform the next step from plan x only (dont pay attention to the other plans). create a code plus a small plan for the next step. when the last step executed, the final output be the final owl ontology that models the story with its restirictions and be able to answer the competency questions. We will later compare the results of the owl files with 'merge' commands at the end (at this stage you need to see them and create a one owl file based on them, it would be like you see the answers of few different ontologist and based on your opinion pick or craete one from them), so dont compare untill the command.
these are the prifixes:
@prefix : <http://www.example.org/test#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
Important: your output should be only owl turtle. Don't write any explanation before the code block or after. Also the code must have 0 comments.
end of footer
Common mistake:
1- plans are identical, try to think differently when generating plans. For producing different plans, you can take three of these approaches and create plans based on them:Bottom-Up Plan, Top-Down Plan,,Agile Plan, Waterfall Plan, Incremental Plan, Spiral Plan, Prototype Plan, Evolutionary Plan, RAD (Rapid Application Development) Plan, Scrum Plan
2- important: when i ask to execute plan x, step y, your output at executing steps must be only codes not explanations.
3- producing output not based on the story and its competency questions
here is one story to know what they look like (The final ontology is about this story and its competency questions):
{story}
End of story
'''

In [ ]:
delete_logfile()
plans = chat_with_gpt(graph_ofT.format(story=Story),temp = 0.5,penalty=0.5)
print(plans)

# plans

[[step 1: Identify the main classes and hierarchies in the ontology],
[step 2: Create object properties to connect the classes],
[step 3: Create data properties to define attributes of the classes]]

[[step 1: Identify the main classes and hierarchies in the ontology],
[step 2: Create object properties to connect the classes],
[step 3: Create reification classes for complex scenarios]]

[[step 1: Identify the main classes and hierarchies in the ontology],
[step 2: Create object properties to connect the classes],
[step 3: Create pivot (reification) classes for complex scenarios]]


In [ ]:
results = []
delete_logfile()
count_error = 0
len_plan = [3,3,3] 1/0
for i in [0,1,2]:
  print('plan in process: ',i+1)
  plan = plans[i]
  History = graph_ofT.format(story=Story) + '\nNow here are the plans you created, in the following, we ask you which step to perform' +\
   'GPT output (plans and steps):' +  '\nplan 1: ' + '\n'.join(plans[0]) + '\nplan 2: ' + '\n'.join(plans[1]) + '\nplan 3: ' + '\n'.join(plans[2])
  for j in range(len_plan[i]):
    try: #if any error happend



      print(f'plan in process: {i+1} step #{j+1}')
      time.sleep(5)
      History += f'\nUser command: Now perform/execute plan {i+1}, step {j+1} by providing the code (turtle syntax) of this step (you must output a code only)'
      output = chat_with_gpt(History)
      # print(output)
      History = History + '\nGPT:'+output+'\n'
      write_to_log(output)



    except:

      count_error += 1
      if count_error > 2:
        1/0
      j = j - 1
      continue

  output = chat_with_gpt(History+f'\nUser command: Now what is the final code output of plan {i+1}  (use turtle syntax and combine all steps into one output), dont miss anything')
  results.append(output)
  write_to_log(History)
  write_to_log(output)


plan in process:  1
plan in process: 1 step #1
plan in process: 1 step #2
plan in process: 1 step #3
plan in process:  2
plan in process: 2 step #1
plan in process: 2 step #2
plan in process: 2 step #3
plan in process:  3
plan in process: 3 step #1
plan in process: 3 step #2
plan in process: 3 step #3


In [ ]:
print('*******************\noutput of plan 1:')
print(results[0])
print('*******************\noutput of plan 2:')
print(results[1])
print('*******************\noutput of plan 3:')
print(results[2])

*******************
output of plan 1:
@prefix : <http://www.example.org/test#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# Step 1: Define classes
:Cl_TheatreFestival a rdfs:Class .
:Cl_Play a rdfs:Class .
:Cl_Location a rdfs:Class .
:Cl_Performance a rdfs:Class .

# Step 2: Define properties
:hasPlay a owl:ObjectProperty ;
    rdfs:domain :Cl_TheatreFestival ;
    rdfs:range :Cl_Play .

:hasLocation a owl:ObjectProperty ;
    rdfs:domain :Cl_TheatreFestival ;
    rdfs:range :Cl_Location .

:hasPerformance a owl:ObjectProperty ;
    rdfs:domain :Cl_TheatreFestival ;
    rdfs:range :Cl_Performance .

# Step 3: Define data properties
:duration a owl:DatatypeProperty ;
    rdfs:domain :Cl_TheatreFestival ;
    rdfs:range xsd:integer .

# Step 4: Define hierarchy
:Cl_LargeFestival rdfs:subClassOf :Cl_TheatreFestival .

# Step 5: Define restrictions
:Cl_LargeFestiv

In [ ]:
results_string = '\noutput of plan 1:\n' + results[0] + '\noutput of plan 2:\n' + results[1] + '\noutput of plan 3:\n' + results[2]
choose = graph_ofT.format(story=Story) + '\nNow here are the plans you created, in the following, we show output of each plan you created in a list of output' + str(plans) +\
  'now you need to combine those three answers into one output in code.\n output of plans:\GPT: ' + results_string
chat_with_gpt(choose)

'@prefix : <http://www.example.org/test#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n### Step 1: Define classes ###\n:Cl_TheatreFestival a rdfs:Class .\n:Cl_Play a rdfs:Class .\n:Cl_Location a rdfs:Class .\n:Cl_Performance a rdfs:Class .\n:Cl_CuratedEvent a rdfs:Class .\n:Cl_SeriesOfEvents a rdfs:Class .\n:Cl_TheatricalPerformance a rdfs:Class .\n:Cl_PlaysInFestival a rdfs:Class .\n:Cl_LargeFestival a rdfs:Class .\n\n### Step 2: Define class hierarchy ###\n:Cl_Play rdfs:subClassOf :Cl_TheatricalPerformance .\n:Cl_Location rdfs:subClassOf :Cl_TheatreFestival .\n:Cl_Performance rdfs:subClassOf :Cl_TheatreFestival .\n:Cl_CuratedEvent rdfs:subClassOf :Cl_TheatreFestival .\n:Cl_SeriesOfEvents rdfs:subClassOf :Cl_TheatreFestival .\n:Cl_TheatricalPerformance rdfs:subClassOf :Cl_TheatreFestival .\n:Cl_PlaysInFestival rdfs:subClassOf :Cl_TheatreFestival .\n:Cl_Lar

## mian experiemtn

In [ ]:
delete_logfile()
S,CQs = stories.story_theatre,stories.story_theatre_cqs
Story = S+'\n'.join(CQs)
plans = chat_with_gpt(graph_ofT.format(story=Story),temp = 0.5,penalty=0.5)
print(plans)

[
["Step 1: Identify main classes from the story such as Theatre, Festival, Production, Play, Actor, Role, City, Country",
 "Step 2: Create subclasses for identified classes if necessary. For example, under Actor class we can have GuestActor and RegularActor",
 "Step 3: Define properties for each class. For instance, 'hasPremier' for Production class or 'worksFor' for Actor class",
 "Step 4: Establish relationships between classes using object properties. For example, a Festival 'takesPlaceIn' a City",
 "Step 5: Apply reification where necessary to capture complex relationships. For instance, an Actor 'playsRoleIn' a Production at a certain point in time",
 "Step 6: Define data properties for each class to capture additional information like 'starRating' of a Production or 'startDate' of an Actor's employment"],

["Step 1: Start by identifying the main entities in the story and create corresponding classes such as Playwright (for William Shakespeare), Review (for Il Gazzettino's review

In [ ]:
results = []
delete_logfile()
count_error = 0
len_plan = [6,6,6] 1/0
for i in [0,1,2]:
  print('plan in process: ',i+1)
  plan = plans[i]
  History = graph_ofT.format(story=Story) + '\nNow here are the plans you created, in the following, we ask you which step to perform' +\
   'GPT output (plans and steps):' +  '\nplan 1: ' + '\n'.join(plans[0]) + '\nplan 2: ' + '\n'.join(plans[1]) + '\nplan 3: ' + '\n'.join(plans[2])
  for j in range(len_plan[i]):
    try: #if any error happend
      print(f'plan in process: {i+1} step #{j+1}')
      time.sleep(5)
      History += f'\nUser command: Now perform/execute plan {i+1}, step {j+1} by providing the code (turtle syntax) of this step (you must output a code only)'
      output = chat_with_gpt(History)
      # print(output)
      History = History + '\nGPT:'+output+'\n'
      write_to_log(output)
    except:
      count_error += 1
      if count_error > 2:
        1/0
      j = j - 1
      continue
  output = chat_with_gpt(History+f'\nUser command: Now what is the final code output of plan {i+1}  (use turtle syntax and combine all steps into one output), dont miss anything')
  results.append(output)
  write_to_log(History)
  write_to_log(output)

plan in process:  1
plan in process: 1 step #1
plan in process: 1 step #2
plan in process: 1 step #3
plan in process: 1 step #4
plan in process: 1 step #5
plan in process: 1 step #6
plan in process:  2
plan in process: 2 step #1
plan in process: 2 step #2
plan in process: 2 step #3
plan in process: 2 step #4
plan in process: 2 step #5
plan in process: 2 step #6
plan in process:  3
plan in process: 3 step #1
plan in process: 3 step #2
plan in process: 3 step #3
plan in process: 3 step #4
plan in process: 3 step #5
plan in process: 3 step #6


In [ ]:
results[0]+'\n**********************\n'+results[1]+'\n**********************\n'+results[2]

'@prefix : <http://www.example.org/test#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n:Cl_Theatre a rdfs:Class .\n:Cl_Festival a rdfs:Class .\n:Cl_Play a rdfs:Class .\n:Cl_Actor a rdfs:Class .\n:Cl_Production a rdfs:Class .\n:Cl_Review a rdfs:Class .\n:Cl_Role a rdfs:Class .\n:Cl_City a rdfs:Class .\n:Cl_Country a rdfs:Class .\n\n:Cl_TheatreFestival a rdfs:Class ;\n    rdfs:subClassOf :Cl_Festival .\n:Cl_TheatreProduction a rdfs:Class ;\n    rdfs:subClassOf :Cl_Production .\n:Cl_TheatreRole a rdfs:Class ;\n    rdfs:subClassOf :Cl_Role .\n:Cl_TheatreReview a rdfs:Class ;\n    rdfs:subClassOf :Cl_Review .\n:Cl_TheatrePlay a rdfs:Class ;\n    rdfs:subClassOf :Cl_Play .\n:Cl_TheatreActor a rdfs:Class ;\n    rdfs:subClassOf :Cl_Actor .\n\n:hasTheatre a rdf:Property ;\n    rdfs:domain :Cl_City ;\n    rdfs:range :Cl_Theatre .\n\n:hasFestival a rdf:Property ;\n   

In [ ]:
results_string = '\noutput of plan 1:\n' + results[0] + '\noutput of plan 2:\n' + results[1] + '\noutput of plan 3:\n' + results[2]
choose = graph_ofT.format(story=Story) + '\nNow here are the plans you created, in the following, we show output of each plan you created in a list of output' + str(plans) +\
  'important: now you need to combine those three answers into one output in code.\n code output of combination of plans:\GPT: ' + results_string
chat_with_gpt(choose)

'["Step 1: Identify main classes from the story such as Theatre, Festival, Production, Play, Actor, Role, City, Country",\n "Step 2: Create subclasses for identified classes if necessary. For example, under Actor class we can have GuestActor and RegularActor",\n "Step 3: Define properties for each class. For instance, \'hasPremier\' for Production class or \'worksFor\' for Actor class",\n "Step 4: Establish relationships between classes using object properties. For example, a Festival \'takesPlaceIn\' a City",\n "Step 5: Apply reification where necessary to capture complex relationships. For instance, an Actor \'playsRoleIn\' a Production at a certain point in time",\n "Step 6: Define data properties for each class to capture additional information like \'starRating\' of a Production or \'startDate\' of an Actor\'s employment"],\n\n["Step 1: Start by identifying the main entities in the story and create corresponding classes such as Playwright (for William Shakespeare), Review (for Il 

In [ ]:
choose

'Your task is to contribute in creation of a well-structured ontology informations that appeared in the given story, requirements, and restrictions (if there are any). You should only provide few diffierent plans that each contains steps to solve this task not actually solving it. Imagin yourself as a prompter of chatGPT and we want to use tree of thoughts. So in this step only create few plans with steps to solve the task. the way you approach this is that you create only a code block in the output that contains a list of  list of strings. Each list is a different plan to solve the problem from step 1 to end, and each string in every list is a thought. the first string is the first thought/step and so on. do not output any other comment or explanation. here are some definisions of ontology components to help you understand the problem in a better way:\nCClasses are the keywords/classes that are going to be node types in the knowledge graph ontology. try to extract all classes, in addi

# CQbyCQ Main Experiment

In [ ]:
cqbycq_prompt = '''Your task is to contribute in creation of a well-structured ontology informations that appeared in the given story, requirements, and restrictions (if there are any).
The way you approach this is first you pick this competency question number "{CQ}" and read the given turtle RDF (we append the code at the end of the prevoius one) to know what is the current ontology till this stage (it can be empty at the begining). Then you add or change the RDF so it can answer to this competency question. Your output at each stage is an append to the previous ones, just do not repeat. You only need to solve the question number so do not touch the next questions since they belong to the next stages of development. you can read this definisions to understand the concepts:
lasses are the keywords/classes that are going to be node types in the knowledge graph ontology. try to extract all classes, in addition, classes are also can be defined for reification. We use Turtle Syntax for representation. Hierarchies are rdfs:subClassOf in the turtle syntax. They can be used to classify similar classes in one superclass. To do this you can find similar nodes and create/use a class as their parent class, for example, adding the node "Cl_employee" is a good middleware and superclass for "Cl_Professors" and "Cl_Administrator" if the story is about modeling ontology of a university. Mostly the lengthier the hierarchy the better. One way can be categorizing classes into several classes and creating superclasses for them. Important: Class names have Cl_ as the prefix for example Cl_Professors. Also keep in mind you can add Equivalent To, General class axioms, Disjoint with, and Disjoint Union of, for each class.
In your ontology modeling, for each competency question, when faced with complex scenarios that involve more than two entities or a combination of entities and datatypes, apply reification. Specifically, create a pivot class to act as an intermediary for these entities, ensuring the nuanced relationships are accurately captured. For instance, when representing "a user accessed a resource at a given time", establish a pivot class like Cl_UserResourceUsage, linked from the user, resource, and the specific time of access to Cl_UserResourceInteraction, rather than directly connecting the user to both the resource and time.
Then you need to create properties (owl:Property). In this step, you use classes from the previous stage and create object and data properties to connect them and establish the ontology. Always output a turtle syntax, if you need more classes to model a competency question between more than 2 concepts, feel free to add more pivot (reification) classes here. try to find as much relation as possible by reading competency questions, restrictions, and stories. At this stage, you can create both data and object properties. Data properties are between classes or hierarchy classes and data types such as xsd:string, xsd:integer, xsd:decimal, xsd:dateTime, xsd:date, xsd:time, xsd:boolean, xsd:byte, xsd:double, xsd:float and etc. For example, in the university domain, we have: employee_id a owl:Property ; rdfs:domain :cl_teacher ; rdfs:range xsd:integer. Object properties are between classes. try to find as much relation as possible by reading competency questions and the story. Feel free to use rdfs:subPropertyOf for creating hierarchies for relations. For modeling properties (object or data properties) if it is necessary, use these relations characteristics: Functional, Inverse functional, Transitive, Symmetric, Asymmetric, Reflexive, and Irreflexive. Also, you are flexible in domain and range so you can use Cl_class1 or Cl_class2 in domain and range or disjoint with, the inverse of between relations.
It is common to forget to add relations that are related to reification: In RDF reification, achieving precise modeling is pivotal, especially when handling multifaceted scenarios where mere binary associations fall short. Take for instance the statement, "a user used a resource at a time". While it might initially seem to involve a direct link between a 'user' and a 'resource', it inherently embodies three entities: a 'user', a 'resource', and a 'time'. Directly connecting 'user' to both 'resource' and 'time' fails to capture the essence of the statement, as it obscures which resource was utilized by the user at a specific time. To address this, a more sophisticated modeling approach is needed, invoking a pivot class, Cl_usingResource. This pivot class acts as an intermediary, linking both Cl_user and Cl_resource. Furthermore, it integrates a time property to denote the exact instance of usage. By employing this method, we can coherently model the statement, ensuring that the user's interaction with a specific resource at a distinct time is unambiguously represented. This approach highlights the imperative of ontology design patterns and the necessity of intermediary nodes when modeling complex relationships involving multiple entities or a mix of entities and datatypes.
Upon implementation of restrictions, feel free to use owl:equivalentClass [ rdf:type owl:Restriction ;  owl:onProperty :{{relation}} ;  owl:allValuesFrom :{{Class}} ] ; in this way, you can put restrictions for classes such as class Cl_C1 is the only class that uses the relation R. or you can put soft restrictions by using owl:someValuesFrom. Also, you can use general class axioms: [ rdf:type owl:Restriction ; owl:onProperty :R1 ; owl:someValuesFrom :Cl_1 ; rdfs:subClassOf :Cl_2 ] when you want to put restrictions on the definition of a class based on its relation and the definition is necessary but not enough (if it is enough it would be equivalent to owl:equivalentClass).
{examples}
these are the prifixes:
@prefix : <http://www.example.org/test#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
Important: before writing the owl code, write this text: is this competency question answarable by the previous version of the RDF (given down) or not? (most likely (~90%) is it not answarable). If no, write a refification class for this question if needed. then solve it. if it was answarable, simply rewrite the given rdf in the output without changing it.
here is one story:
{story}
End of story
here are some possible mistakes that you might make:
1- forgetting to add prefixes at the beginning of the code.
2- forgetting to write pivot classes at the beginning before starting to code.
3- your output would be concatinated to the prevoius output rdf, so dont write repititive words, classes or ...
4- in your output put all of the previous rdf classes, relations, restrictions and add yours. your output would be passed to the next stage so dont remove previous code (it is going to replace the previous rdf)
5- you usually forget to write the name of the reification (pivot) that you want create at the begining of the output
6- In reification, the reification node (pivot class) is connected to all related classes by object properties not by subclass of. it can be subclass of something, but for reification it need object properties.
common mistakes in extracting classes:
1- mistake: not extracting all classes and missing many of them. classes can be found in the story, or in the competency question number and restrictions.
2- Returning empty answer
3- Providing comments or explanations
4- Extracint classes like 'Date', and 'integer' are wrong since they are data properties.
5- not using RDF reification: not extracting pivot classes for modeling relation between classes (more than one class and one dataproperty, or more than two classes)
6- extracting individuals in the text as class
7- pivot class is not sublcass of its components.
common mistakes in the hierarchy extraction:
1- creating an ontology for non-existing classes: creating a new leaf and expanding it into the root
2- returning empty answer or very short
3- Providing comments or explanations
4- Extracting attributes such as date, time, and string names that are related to data properties
5- Forget to add "" around the strings in the tuples
Common mistakes in the object_properties:
1- returning new variables with anything except object_properties
2- returning empty answer or very short
3- providing comments or explanations
4- when the pivot class is created, all of the related classes should point to it (direction of relation is from the classes (domains) 'to'  pivot class (range))
Common mistakes in the data_properties:
1- returning new variables with anything except data_properties
2- returning empty answer or very short
3- providing comments or explanations
Here is the last RDF:
{rdf}
'''
examples = '''
examples on how to solve competency questions:
Pattern Name: Submissions:Situation

Short Description:
This is designed to represent contexts or situations, along with the entities or elements that are contextualized within these situations. It aims to capture the relational context of entities, allowing for a structured representation of situations and their associated elements.
Competency Questions:
What is the context or situation of a particular entity?
What entities or elements are present in a specific context or situation?
Simplified Code:
code:
:hasSetting rdf:type owl:ObjectProperty ;
    rdfs:domain :SituationReification ;
    rdfs:range :Situation.
:Situation rdf:type owl:Class .
:SituationReification rdf:type owl:Class .
--------
Pattern Name: Submissions:AgentRole
Short Description:
This is designed to represent agents and the roles they play. It focuses on the relationship between agents (entities capable of action) and their roles in various contexts, allowing for a structured representation of this relationship.
Competency Questions:
Which agent plays this role?
What is the role played by a particular agent?
code:
:Agent rdf:type owl:Class ;
    rdfs:subClassOf :Object;
    owl:disjointWith :Role.
:Object rdf:type owl:Class .
:Role rdf:type owl:Class .
--------
Pattern Name: Submissions:Bag
Short Description:
Competency Questions:
What bag is a particular item an element of?
What is the size of a specific bag?
What resource does a particular item refer to?
What are the items contained in a given bag?
code:
:Bag rdf:type owl:Class ;
    rdfs:subClassOf :Collection ;
:Item rdf:type owl:Class ;
    rdfs:subClassOf owl:Thing ;
    rdfs:comment "Item - Element belonging to a Bag".
:itemContent rdf:type owl:ObjectProperty ;
    rdfs:domain :Item ;
    rdfs:range owl:Thing ;
:itemOf rdf:type owl:ObjectProperty ;
    rdfs:domain :Item ;
    rdfs:range :Bag ;
:hasItem rdf:type owl:ObjectProperty ;
    rdfs:domain :Bag ;
    rdfs:range :Item ;
:size rdf:type owl:DatatypeProperty ;
    rdfs:domain :Collection ;
    rdfs:range xsd:integer ;
---------
Pattern Name: Submissions:Classification
Short Description:
Competency Questions:
What concept is assigned to this entity?
Which category does this entity belong to?
Simplified Code:
:Concept rdf:type owl:Class .
:classifies rdf:type owl:ObjectProperty ;
    rdfs:domain :Concept ;
    rdfs:range owl:Thing.
:isClassifiedBy rdf:type owl:ObjectProperty ;
    rdfs:domain owl:Thing ;
    rdfs:range :Concept.
---------
Pattern Name: CourseInformation
Short Description:
The 'CourseInformation' ontology design pattern is tailored to model the essential attributes and relationships of a course within an educational institution. It focuses on capturing the key details of courses, their hierarchical structure, and their association with students and instructors.
Competency Questions:
What is the course name?
What is the course number?
Who is the instructor of the course?
Who is taking the course?
:Course rdf:type owl:Class .
:Grad-Course rdf:type owl:Class ;
    rdfs:subClassOf :Course .
:Under-Grad-Course rdf:type owl:Class ;
    rdfs:subClassOf :Course .
:Student rdf:type owl:Class .
:Grad-Student rdf:type owl:Class ;
    rdfs:subClassOf :Student .
:Under-Grad-Student rdf:type owl:Class ;
    rdfs:subClassOf :Student .
:Instructor rdf:type owl:Class .
:teaches rdf:type owl:ObjectProperty ;
    rdfs:domain :Instructor ;
    rdfs:range :Course .
:taughtBy rdf:type owl:ObjectProperty ;
    owl:inverseOf :teaches ;
    rdfs:domain :Course ;
    rdfs:range :Instructor .
:takenBy rdf:type owl:ObjectProperty ;
    rdfs:domain :Course ;
    rdfs:range :Student .
----------
Pattern Name: Submissions:Description
Short Description:
To formally represent a conceptualization or a descriptive context.
Competency Questions:
Which are the assumptions under which a certain thing is described?
Which are the concepts involved in the description of a certain thing?
What is the interpretation of a specific case, event, or observation?
:descriptionDefines rdf:type owl:ObjectProperty ;
    rdfs:label "defines" ;
    rdfs:domain :Description ;
    rdfs:range :Concept.
:descriptionIsDefinedIn rdf:type owl:ObjectProperty ;
    rdfs:label "is defined in" ;
    rdfs:domain :Concept ;
    rdfs:range :Description.
:descriptionUsesConcept rdf:type owl:ObjectProperty ;
    rdfs:label "uses concept" ;
    rdfs:domain :Description ;
    rdfs:range :Concept.
:Concept rdf:type owl:Class ;
    rdfs:label "Concept" .
:Description rdf:type owl:Class ;
    rdfs:label "Description" .
--------
Pattern Name: Submissions:Region

Short Description:
This ontology design pattern, named 'Region', is designed to represent and reason about the values of attributes of things by explicitly considering the dimensions or "regions" of these attributes, which include those values. It facilitates understanding and reasoning about various attributes and their associated values in a structured manner.
Competency Questions:
What is the value for the attribute of a particular entity?
Which entities have a certain value on a specific parameter, attribute, or feature?
Simplified Code:
:hasRegion rdf:type owl:ObjectProperty ;
    rdfs:domain owl:Thing ;
    rdfs:range :Region.
:isRegionFor rdf:type owl:ObjectProperty ;
    rdfs:domain :Region ;
    rdfs:range owl:Thing.
:hasRegionDataValue rdf:type owl:DatatypeProperty ;
    rdfs:domain :Region.
:Region rdf:type owl:Class .
'''


In [ ]:
def cqbycq(story,cqs,sleep = 3):
  rdf = ''
  err = 0
  for i in range(len(cqs)):
    try:
      print('doing cq #',i+1)
      rdf = rdf+ '\n' + chat_with_gpt(cqbycq_prompt.format(story = story, examples='',CQ = cqs[i],rdf = rdf))
      write_to_log(rdf)
      write_to_log('doing cq # '+str(i+1))

      if i < len(cqs) - 1:
        time.sleep(sleep)
    except:
      err +=1
      i -=1
      if err >5:
        break
      continue
  write_to_log('final output of cqbycq')
  write_to_log(rdf)
  return rdf

In [ ]:
stories = Datasets()
S,CQs = stories.story_theatre,stories.story_theatre_cqs
delete_logfile()

write_to_log('hospital_music')
cqbycq(S,CQs,sleep = 5)


doing cq # 1
doing cq # 2
doing cq # 3
doing cq # 4
doing cq # 5
doing cq # 6
doing cq # 7
doing cq # 8
doing cq # 9
doing cq # 10
doing cq # 11
doing cq # 12
doing cq # 13
doing cq # 14
doing cq # 15


'\nThe competency question "When did a certain theatre festival take place?" is not answerable by the previous version of the RDF as it is not provided.\n\nWe need to create a pivot class for the theatre festival that includes the date of the festival.\n\nClasses:\n- Cl_TheatreFestival\n- Cl_Date\n\nHierarchy:\n- Cl_TheatreFestival rdfs:subClassOf :Cl_Event.\n- Cl_Date rdfs:subClassOf :xsd:date.\n\nObject Properties:\n- :hasDate a owl:ObjectProperty ; rdfs:domain :Cl_TheatreFestival ; rdfs:range :Cl_Date.\n\nData Properties:\n- :dateValue a owl:DatatypeProperty ; rdfs:domain :Cl_Date ; rdfs:range xsd:string.\n\nHere is the updated RDF:\n\n@prefix : <http://www.example.org/test#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n:Cl_TheatreFestival rdf:type owl:Class .\n:Cl_Date rdf:type owl:Class .\n\n:Cl_TheatreFestival rdfs:subClassOf :Cl_Event.\n:Cl_Date rdfs

In [ ]:
stories = Datasets()
S,CQs = stories.story_theatre,stories.story_theatre_cqs
delete_logfile()

write_to_log('hospital_music')
cqbycq(S,CQs,sleep = 5)

doing cq # 1
doing cq # 2
doing cq # 3
doing cq # 4
doing cq # 5
doing cq # 6
doing cq # 7
doing cq # 8
doing cq # 9
doing cq # 10
doing cq # 11
doing cq # 12
doing cq # 13
doing cq # 14
doing cq # 15


'\nThe competency question "When did a certain theatre festival take place?" is not answerable by the previous version of the RDF because there is no previous RDF provided.\n\nTo answer this competency question, we need to create classes for Theatre Festival and Date, and a pivot class for FestivalOccurrence. The FestivalOccurrence class will connect the Theatre Festival and Date classes.\n\nHere is the RDF code:\n\n@prefix : <http://www.example.org/test#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n:Cl_TheatreFestival a rdfs:Class .\n:Cl_Date a rdfs:Class .\n:Cl_FestivalOccurrence a rdfs:Class .\n\n:Cl_FestivalOccurrence rdfs:subClassOf :Cl_TheatreFestival, :Cl_Date .\n\n:Pr_takesPlaceOn a rdf:Property ;\n    rdfs:domain :Cl_FestivalOccurrence ;\n    rdfs:range :Cl_Date .\n\n:Pr_isFestival a rdf:Property ;\n    rdfs:domain :Cl_FestivalOccurrence ;\n    rd

In [ ]:
1==1

True

In [ ]:
stories = Datasets()
S,CQs = stories.story_hospital,stories.story_hospital_cqs


In [ ]:
for i in range(len(CQs)):
  print(i+1,CQs[i])

1 What medical degrees does a certain person have?
2 During what time period did a certain person study for a specific degree?
3 When was a certain person first employed at a certain hospital?
4 In what city is a certain hospital located?
5 In what country is a certain city located?
6 Who are the members of a certain union at a certain point in time?
7 What role does a certain person have within a certain union group at a certain point in time?
8 What is the evaluation statement given by a certain doctor for a certain employee?
9 What articles is a specific book or CD composed of?
10 How many pages does a particular book contain?
11 When was a certain book or CD published?
12 When did a certain seminar take place?
13 What articles were presented in a certain seminar?
14 Where did a certain seminar take place?
15 At least one article is always presented at each seminar.
